In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import sqlite3
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#from wordcloud import WordCloud
import re
import os
from sqlalchemy import create_engine # database connection
import datetime as dt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from skmultilearn.adapt import mlknn
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import LabelPowerset
from sklearn.naive_bayes import GaussianNB
from datetime import datetime

In [2]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from tqdm import tqdm

In [3]:
train_1 = np.load('final_train.npy')


In [4]:
train_1.shape

(9489, 102296)

In [5]:

train_3 = np.load('topic_train.npy')

In [6]:
train_3.shape

(9489, 300)

In [7]:
test_1 = np.load('final_test.npy')

test_3 = np.load('topic_test.npy')

In [8]:
test_1.shape

(2966, 102296)

In [9]:
test_3.shape

(2966, 300)

In [10]:
data = pd.read_csv('data.csv')
data.head()

,Unnamed: 0,imdb_id,title,plot_synopsis,tags,split,synopsis_source,New,cleaned
0,0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb,"find, reveal, leave, discover, real, follow, m...",find reveal leave discover real follow miss mu...
1,1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb,"fight, evil, save, great, free, lead, find, da...",fight evil save great free lead find dark retu...
2,2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb,"shoot, back, drive, make, arrive, give, hit, l...",shoot back drive make arrive give hit leave st...
3,3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb,"play, show, billy, win, include, end, perform,...",play show billy win include end perform make f...
4,4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb,"make, back, give, good, talk, leave, walk, sta...",make back give good talk leave walk start feel...


In [11]:
train_data =  data['split']=='train'
train = data[train_data]

   
    
test_data =  data['split']=='test'
test = data[test_data]

  
    
validation_data =  data['split']=='val'
val = data[validation_data]


In [12]:
vectorizer_tags = CountVectorizer(tokenizer = lambda x: x.split(','), binary='true')
y_train = vectorizer_tags.fit_transform(train['tags'])
y_test = vectorizer_tags.transform(test['tags'])

In [13]:
y_train.shape

(9489, 142)

In [14]:
x_train = np.hstack((train_1, train_3))

In [15]:
x_train.shape

(9489, 102596)

In [16]:
x_test = np.hstack((test_1, test_3))

In [17]:
x_test.shape

(2966, 102596)

In [40]:
from skmultilearn.adapt import MLkNN


start = datetime.now()

classifier = MLkNN(k=20)
classifier.fit(x_train, y_train)
predictions = classifier.predict(x_test)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.1830748482805125
Hamming loss  0.01777421101117833
Micro-average quality numbers
Precision: 0.6994, Recall: 0.2986, F1-measure: 0.4185
Macro-average quality numbers
Precision: 0.4490, Recall: 0.1988, F1-measure: 0.2420
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        35
           1       0.60      0.13      0.21       117
           2       0.92      0.85      0.88        26
           3       0.00      0.00      0.00        11
           4       1.00      0.15      0.27        13
           5       0.73      0.31      0.43        26
           6       1.00      0.07      0.12        15
           7       0.65      0.20      0.31        75
           8       0.00      0.00      0.00         4
           9       0.75      0.58      0.65        31
          10       0.86      0.75      0.80         8
          11       0.00      0.00      0.00        20
          12       0.67      0.18      0.28        79
          1

In [42]:
start = datetime.now()

classifier = MLkNN(k=15)
classifier.fit(x_train, y_train)
predictions = classifier.predict(x_test)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.19318948078219825
Hamming loss  0.01788343004758151
Micro-average quality numbers
Precision: 0.6820, Recall: 0.3095, F1-measure: 0.4257
Macro-average quality numbers
Precision: 0.4631, Recall: 0.2181, F1-measure: 0.2639
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        35
           1       0.62      0.13      0.21       117
           2       0.92      0.85      0.88        26
           3       0.00      0.00      0.00        11
           4       0.75      0.23      0.35        13
           5       0.56      0.38      0.45        26
           6       0.67      0.13      0.22        15
           7       0.84      0.21      0.34        75
           8       0.00      0.00      0.00         4
           9       0.75      0.77      0.76        31
          10       0.86      0.75      0.80         8
          11       0.00      0.00      0.00        20
          12       0.76      0.16      0.27        79
          

In [43]:
start = datetime.now()

classifier = MLkNN(k=10)
classifier.fit(x_train, y_train)
predictions = classifier.predict(x_test)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.1968981793661497
Hamming loss  0.01816360061922445
Micro-average quality numbers
Precision: 0.6551, Recall: 0.3211, F1-measure: 0.4310
Macro-average quality numbers
Precision: 0.4782, Recall: 0.2330, F1-measure: 0.2757
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        35
           1       0.49      0.21      0.30       117
           2       0.89      0.92      0.91        26
           3       0.00      0.00      0.00        11
           4       0.50      0.23      0.32        13
           5       0.52      0.54      0.53        26
           6       0.71      0.33      0.45        15
           7       0.84      0.21      0.34        75
           8       0.00      0.00      0.00         4
           9       0.71      0.71      0.71        31
          10       0.86      0.75      0.80         8
          11       0.00      0.00      0.00        20
          12       0.67      0.15      0.25        79
          1

In [18]:
start = datetime.now()

from skmultilearn.adapt import MLkNN

classifier = MLkNN(k=5)
classifier.fit(x_train, y_train)
predictions = classifier.predict(x_test)

print("Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print (metrics.classification_report(y_test, predictions))
print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.22016183412002696
Hamming loss  0.01835829542324751
Micro-average quality numbers
Precision: 0.6427, Recall: 0.3220, F1-measure: 0.4290
Macro-average quality numbers
Precision: 0.4791, Recall: 0.2346, F1-measure: 0.2852
              precision    recall  f1-score   support

           0       0.09      0.03      0.04        35
           1       0.58      0.18      0.27       117
           2       0.93      0.96      0.94        26
           3       0.00      0.00      0.00        11
           4       0.62      0.38      0.48        13
           5       0.67      0.46      0.55        26
           6       0.75      0.20      0.32        15
           7       0.84      0.21      0.34        75
           8       0.00      0.00      0.00         4
           9       0.72      0.84      0.78        31
          10       0.86      0.75      0.80         8
          11       0.00      0.00      0.00        20
          12       0.67      0.18      0.28        79
          